## Dockerfile

* The dockerfile defines the environment in which our server will be executed. 
* Below, you can see that the entrypoint for our container will be [restrserve.R](restrserve.R)

In [1]:
%pycat Dockerfile

FROM r-base:3.6.3

MAINTAINER Amazon SageMaker Examples <amazon-sagemaker-examples@amazon.com>


RUN R -e "install.packages(c('RestRserve','xgboost','dplyr'), repos='https://cloud.r-project.org')"

COPY xgb.model /opt/ml/xgb.model
COPY restrserve.R /opt/ml/restrserve.R

WORKDIR /opt/ml

ENTRYPOINT ["/usr/bin/Rscript", "/opt/ml/restrserve.R", "--no-save"]


## restrserve.R

**restrserve.R** handles the following steps
* Loads the R libraries used by the server.
* Loads a pretrain xgboost model that has been trained on the classical Iris dataset, [iris.csv](iris.csv).
* Defines an inference function that takes a matrix of iris features and returns predictions for those iris examples.
* defines two routes:
    * `/ping` returns a string 'Alive' to indicate that application is healthy
    * `/invocations` applies the previously defined inference function to the input features from the request body

In [2]:
%pycat restrserve.R

library(xgboost)
library(RestRserve)

# load a pretrained xgboost model
bst <- xgb.load("xgb.model")

# create a closure around our xgboost model and input data processing
inference <- function(x){
  ds <- xgb.DMatrix(data = x )
  output <- predict(bst, ds)
  list(output=output)
}
app = Application$new()

app$add_get(
  path = "/ping",
  FUN = function(request, response) {
    response$set_body(list(Status = "Alive"))
  })

app$add_post(
  path = "/invocations",
  FUN = function(request, response) {
    result = inference(do.call(rbind,request$body$features))
    response$set_content_type("application/json")
    response$set_body(result)
  })


backend = BackendRserve$new()
backend$start(app, http_port = 8080)


## Build the Serving Image

In [3]:
!docker build -t r-restrserve .

Sending build context to Docker daemon  247.8kB
Step 1/7 : FROM r-base:3.6.3
 ---> cec2502269fb
Step 2/7 : MAINTAINER Amazon SageMaker Examples <amazon-sagemaker-examples@amazon.com>
 ---> Using cache
 ---> d5c7ee17124e
Step 3/7 : RUN R -e "install.packages(c('RestRserve','xgboost','dplyr'), repos='https://cloud.r-project.org')"
 ---> Using cache
 ---> ebcf2f81ff2d
Step 4/7 : COPY xgb.model /opt/ml/xgb.model
 ---> Using cache
 ---> d3c15c4582c7
Step 5/7 : COPY restrserve.R /opt/ml/restrserve.R
 ---> Using cache
 ---> 1f157953f1d5
Step 6/7 : WORKDIR /opt/ml
 ---> Using cache
 ---> 7f4145abcde0
Step 7/7 : ENTRYPOINT ["/usr/bin/Rscript", "/opt/ml/restrserve.R", "--no-save"]
 ---> Using cache
 ---> 5de8902faece
Successfully built 5de8902faece
Successfully tagged r-restrserve:latest


## Launch the Serving Container

In [4]:
!echo "Launching RestRServer"
!docker run -d  --rm -p 5000:8080 r-restrserve 
!echo "Waiting for the server to start.." && sleep 10

Launching RestRServer
2c01e23912306de82f39f4d76db6aa263825e9c9b16c97bbcdf74fe306df4022
Waiting for the server to start..


In [5]:
!docker container list

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                    NAMES
2c01e2391230        r-restrserve        "/usr/bin/Rscript /o…"   11 seconds ago      Up 10 seconds       0.0.0.0:5000->8080/tcp   loving_haslett


## Define Simple Python Client

In [6]:
import requests
from tqdm import tqdm
import pandas as pd
pd.set_option('display.max_rows', 500)

In [7]:
def get_predictions(examples, instance=requests, port=5000):
    payload = {"features": examples}
    return instance.post(f"http://127.0.0.1:{port}/invocations", json=payload)

In [8]:
def get_health(instance=requests, port=5000):
    instance.get(f"http://127.0.0.1:{port}/ping")

## Define Example Inputs

Let's define example inputs from the [iris.csv](iris.csv) dataset.

In [9]:
iris = pd.read_csv("iris.csv")

In [10]:
iris_features = iris[["Sepal.Length", "Sepal.Width", "Petal.Length", "Petal.Width"]]

In [11]:
example_inputs = iris_features.values.tolist()

### Plumber

In [12]:
predicted = get_predictions(example_inputs).json()["output"]

In [13]:
iris["predicted"] = predicted

In [14]:
iris

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,predicted
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0
2,4.7,3.2,1.3,0.2,setosa,0
3,4.6,3.1,1.5,0.2,setosa,0
4,5.0,3.6,1.4,0.2,setosa,0
5,5.4,3.9,1.7,0.4,setosa,0
6,4.6,3.4,1.4,0.3,setosa,0
7,5.0,3.4,1.5,0.2,setosa,0
8,4.4,2.9,1.4,0.2,setosa,0
9,4.9,3.1,1.5,0.1,setosa,0


### Stop All Serving Containers

Finally, let's shutdown the serving container we launched for the test.

In [15]:
!docker kill $(docker ps -q)

2c01e2391230
